<a target="_blank" href="https://colab.research.google.com/github/Nicopiwi/eliminatorias_conmebol_ple_argentina/blob/main/eliminatorias.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# Modelo

## Variables

$$
\delta_k = 1 \ \text{sii la k-esima combinacion se cumple}
$$

(Para que Argentina no tenga garantizada la clasificación, debe haber alguna forma para que alguna combinación de 6 países terminen por arriba de Argentina. 
En este caso, tenemos  $\binom{9}{6} = 84$ combinaciones).

$$
home_i = 1 \ \text{sii el equipo local del i-esimo partido gana}
$$

$$
away_i = 1 \ \text{sii el equipo visitante del i-esimo partido gana}
$$

$$
puntaje_c = \text{Puntaje final del país c}
$$

## Problema

$$
max \ \delta_1 + ... + \delta_{84} \\
$$

Sujeto a

$$
away_i = 1 \ \forall i \in locales(Argentina)
$$

$$
home_i = 1 \ \forall i \in visitantes(Argentina)
$$

"Argentina pierde de ahora en más"

$$
home_i + away_i + draw_i = 1 \ \forall i \in partidos
$$

"Consistencia de variables en cada partido"

$$
puntaje_c = 3 \sum_{i \in locales(c)} home_i + 3 \sum_{i \in visitantes(c)} away_i + \sum_{locales(c) \ \cup \ visitantes(c)} draw_i + PuntajeInicial(c)
$$

"Definición de la variable $puntaje_c$"

$$
puntaje_{argentina} < puntaje_c + M(1-\delta_k) \ \forall k \in Combinaciones, \ \forall c \in PaisesCombinacion(k)
$$

"Activación de la variable $\delta_k$ si la k-ésima combinación de países supera a Argentina al final"


In [10]:
import cplex
import pandas as pd


def crear_y_correr_modelo(puntaje_inicial_argentina=None, forzar_proximos_partidos_dict=None):
    # Cargar los datos
    partidos = pd.read_csv('partidos.csv')  # Contiene info sobre partidos locales y visitantes
    puntajes_iniciales = pd.read_csv('puntajes_iniciales.csv')  # Puntajes iniciales de los países
    combinaciones = pd.read_csv('combinaciones.csv')  # Combinaciones de países

    if puntaje_inicial_argentina is None:
        puntaje_inicial_argentina = float(puntajes_iniciales[puntajes_iniciales['id'] == 1]['puntaje'].values[0])

    # Obtener la lista de equipos y partidos
    equipos = puntajes_iniciales['id'].unique()
    partidos_ids = set(partidos["id"].unique().tolist())

    # Inicializar el modelo de CPLEX
    modelo = cplex.Cplex()
    argentina_idx = 1

    # Definir las variables de decisión
    home_vars = [f"home_{i}" for i in partidos_ids]
    away_vars = [f"away_{i}" for i in partidos_ids]
    draw_vars = [f"draw_{i}" for i in partidos_ids]
    puntajes_vars = [f"puntaje_{c}" for c in equipos]
    delta_vars = [f"delta_{k}" for k in combinaciones['id']]

    # Añadir las variables al modelo
    modelo.variables.add(names=home_vars, lb=[0] * len(home_vars), ub=[1] * len(home_vars), types=['B'] * len(home_vars))
    modelo.variables.add(names=away_vars, lb=[0] * len(away_vars), ub=[1] * len(away_vars), types=['B'] * len(away_vars))
    modelo.variables.add(names=draw_vars, lb=[0] * len(draw_vars), ub=[1] * len(draw_vars), types=['B'] * len(draw_vars))
    modelo.variables.add(names=puntajes_vars, lb=[0] * len(puntajes_vars), ub=[1000] * len(puntajes_vars), types=['C'] * len(puntajes_vars))
    modelo.variables.add(names=delta_vars, lb=[0] * len(delta_vars), ub=[1] * len(delta_vars), types=['B'] * len(delta_vars))

    # Función objetivo: maximizar delta_1
    modelo.objective.set_sense(modelo.objective.sense.maximize)
    terms = []

    for k in combinaciones['id']:
        terms.append((f"delta_{k}", 1))

    modelo.objective.set_linear(terms)

    # Restricciones
    locales_argentina = partidos[(partidos['home_id'] == argentina_idx)]["id"]
    for i in locales_argentina:
        if forzar_proximos_partidos_dict is not None and int(i) in forzar_proximos_partidos_dict.keys():
            if forzar_proximos_partidos_dict[i] == 'win':
                modelo.linear_constraints.add(
                    lin_expr=[cplex.SparsePair(ind=[f"home_{i}"], val=[1])],
                    senses=['E'],
                    rhs=[1]
                )
            elif forzar_proximos_partidos_dict[i] == 'lose':
                modelo.linear_constraints.add(
                    lin_expr=[cplex.SparsePair(ind=[f"away_{i}"], val=[1])],
                    senses=['E'],
                    rhs=[1]
                )
            else:
                modelo.linear_constraints.add(
                    lin_expr=[cplex.SparsePair(ind=[f"draw_{i}"], val=[1])],
                    senses=['E'],
                    rhs=[1]
                )
        else:
            modelo.linear_constraints.add(
                lin_expr=[cplex.SparsePair(ind=[f"away_{i}"], val=[1])],
                senses=['E'], 
                rhs=[1]
            )

    visitantes_argentina = partidos[(partidos['away_id'] == argentina_idx)]["id"]
    for i in visitantes_argentina:

        if forzar_proximos_partidos_dict is not None and int(i) in forzar_proximos_partidos_dict.keys():
            if forzar_proximos_partidos_dict[i] == 'win':
                modelo.linear_constraints.add(
                    lin_expr=[cplex.SparsePair(ind=[f"away_{i}"], val=[1])],
                    senses=['E'],
                    rhs=[1]
                )
            elif forzar_proximos_partidos_dict[i] == 'lose':
                modelo.linear_constraints.add(
                    lin_expr=[cplex.SparsePair(ind=[f"home_{i}"], val=[1])],
                    senses=['E'],
                    rhs=[1]
                )
            else:
                modelo.linear_constraints.add(
                    lin_expr=[cplex.SparsePair(ind=[f"draw_{i}"], val=[1])],
                    senses=['E'],
                    rhs=[1]
                )

        else:
            modelo.linear_constraints.add(
                lin_expr=[cplex.SparsePair(ind=[f"home_{i}"], val=[1])],
                senses=['E'],
                rhs=[1]
            )

    for i in partidos_ids:
        modelo.linear_constraints.add(
            lin_expr=[cplex.SparsePair(ind=[f"home_{i}", f"away_{i}", f"draw_{i}"], val=[1, 1, 1])],
            senses=['E'],
            rhs=[1]
        )


    for c in equipos:
        ind = [f"puntaje_{c}"]
        val = [1]

        # Calcular puntaje de cada país
        partidos_locales = set(partidos[(partidos['home_id'] == c)]["id"].tolist())

        for i in partidos_locales:
            ind.append(f"home_{i}")
            val.append(-3)
        
        partidos_visitantes = set(partidos[(partidos['away_id'] == c)]["id"].tolist())
        for i in partidos_visitantes:
            ind.append(f"away_{i}")
            val.append(-3)

        for i in partidos_locales.union(partidos_visitantes):
            ind.append(f"draw_{i}")
            val.append(-1)

        if c == argentina_idx:
            modelo.linear_constraints.add(
                lin_expr=[cplex.SparsePair(ind=ind, val=val)],
                senses=['E'],
                rhs=[float(puntaje_inicial_argentina)]
            )
        else:
            modelo.linear_constraints.add(
                lin_expr=[cplex.SparsePair(ind=ind, val=val)],
                senses=['E'],
                rhs=[float(puntajes_iniciales[puntajes_iniciales['id'] == c]['puntaje'].values[0])]
            )

    M = 1000  # Constante grande para el modelo
    for _, comb in combinaciones.iterrows():
        k = comb['id']
        paises_k = comb[1:].dropna().values
        for c in paises_k:
            modelo.linear_constraints.add(
                lin_expr=[cplex.SparsePair(
                    ind=[f"puntaje_{argentina_idx}", f"puntaje_{c}", f"delta_{k}"],
                    val=[1, -1, M]
                )],
                senses=['L'],
                rhs=[M]
            )
    modelo.solve()

    # modelo.write('debug.lp')

    sol = modelo.solution
    print(f"Solución óptima: {sol.get_objective_value()}")
    for i in partidos_ids:
        print(f"Partido {i}: home={sol.get_values(f'home_{i}')}, away={sol.get_values(f'away_{i}')}, draw={sol.get_values(f'draw_{i}')}")
    for k in combinaciones['id']:
        print(f"Combinación {k}: delta={sol.get_values(f'delta_{k}')}")

    values_deltas = [sol.get_values(f'delta_{k}') for k in combinaciones['id']]

    if any([v == 1 for v in values_deltas]):
        print("\n Argentina todavía no tiene garantizado el pase al mundial")
    else:
        print("\n Argentina tiene garantizado el pase al mundial :)))")



## Evaluación puntajes actuales

In [12]:
crear_y_correr_modelo()

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve eliminated 74 rows and 26 columns.
MIP Presolve modified 1904 coefficients.
Reduced MIP has 488 rows, 188 columns, and 1114 nonzeros.
Reduced MIP has 180 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (1.15 ticks)
Found incumbent of value 0.000000 after 0.02 sec. (1.41 ticks)
Probing time = 0.00 sec. (0.78 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 488 rows, 188 columns, and 1114 nonzeros.
Reduced MIP has 180 binaries, 8 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.85 ticks)
Probing time = 0.00 sec. (0.78 ticks)
Clique table members: 32.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 16 threads.
Root relaxation solution time = 0.00 sec. (1.49 ticks)

        Nodes                             

## Mínimo puntaje que tendría que tener Argentina para estar clasificado



In [3]:
puntaje_inicial = 32
crear_y_correr_modelo(puntaje_inicial_argentina=puntaje_inicial)

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve eliminated 482 rows and 107 columns.
MIP Presolve modified 2016 coefficients.
Reduced MIP has 80 rows, 107 columns, and 284 nonzeros.
Reduced MIP has 100 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (1.00 ticks)
Found incumbent of value 0.000000 after 0.00 sec. (1.14 ticks)
Probing time = 0.00 sec. (0.07 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 80 rows, 107 columns, and 284 nonzeros.
Reduced MIP has 100 binaries, 7 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.21 ticks)
Probing time = 0.00 sec. (0.07 ticks)
Clique table members: 31.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 16 threads.
Root relaxation solution time = 0.00 sec. (0.44 ticks)

        Nodes                               

Default row names c1, c2 ... being created.


Solución óptima: 0.0
Partido 1: home=1.0, away=0.0, draw=0.0
Partido 2: home=1.0, away=0.0, draw=0.0
Partido 3: home=1.0, away=0.0, draw=0.0
Partido 4: home=1.0, away=0.0, draw=0.0
Partido 5: home=1.0, away=0.0, draw=0.0
Partido 6: home=1.0, away=0.0, draw=0.0
Partido 7: home=1.0, away=0.0, draw=0.0
Partido 8: home=0.0, away=1.0, draw=0.0
Partido 9: home=0.0, away=1.0, draw=0.0
Partido 10: home=1.0, away=0.0, draw=0.0
Partido 11: home=1.0, away=0.0, draw=0.0
Partido 12: home=1.0, away=0.0, draw=0.0
Partido 13: home=1.0, away=0.0, draw=0.0
Partido 14: home=0.0, away=1.0, draw=0.0
Partido 15: home=1.0, away=0.0, draw=0.0
Partido 16: home=1.0, away=0.0, draw=0.0
Partido 17: home=1.0, away=0.0, draw=0.0
Partido 18: home=1.0, away=0.0, draw=0.0
Partido 19: home=0.0, away=1.0, draw=0.0
Partido 20: home=0.0, away=1.0, draw=0.0
Partido 21: home=1.0, away=0.0, draw=0.0
Partido 22: home=1.0, away=0.0, draw=0.0
Partido 23: home=1.0, away=0.0, draw=0.0
Partido 24: home=1.0, away=0.0, draw=0.0
Part

## Y si Argentina gana el próximo partido?

In [4]:
crear_y_correr_modelo(forzar_proximos_partidos_dict={2: 'win'})

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve eliminated 17 rows and 25 columns.
MIP Presolve modified 2016 coefficients.
Reduced MIP has 545 rows, 189 columns, and 1241 nonzeros.
Reduced MIP has 180 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (1.21 ticks)
Found incumbent of value 0.000000 after 0.02 sec. (1.48 ticks)
Probing time = 0.00 sec. (0.88 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 545 rows, 189 columns, and 1241 nonzeros.
Reduced MIP has 180 binaries, 9 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.92 ticks)
Probing time = 0.00 sec. (0.88 ticks)
Clique table members: 32.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 16 threads.
Root relaxation solution time = 0.00 sec. (1.55 ticks)

        Nodes                             

Default row names c1, c2 ... being created.


Solución óptima: 84.0
Partido 1: home=1.0, away=0.0, draw=-0.0
Partido 2: home=0.0, away=1.0, draw=0.0
Partido 3: home=0.0, away=1.0, draw=-0.0
Partido 4: home=1.0, away=0.0, draw=-0.0
Partido 5: home=1.0000000000000007, away=0.0, draw=-0.0
Partido 6: home=1.0, away=0.0, draw=-0.0
Partido 7: home=0.0, away=1.0, draw=-0.0
Partido 8: home=0.0, away=1.0, draw=0.0
Partido 9: home=1.0, away=0.0, draw=-0.0
Partido 10: home=0.0, away=1.0, draw=-0.0
Partido 11: home=1.0, away=0.0, draw=-0.0
Partido 12: home=0.0, away=1.0, draw=-0.0
Partido 13: home=0.0, away=1.0, draw=-0.0
Partido 14: home=0.0, away=-0.0, draw=1.0000000000000013
Partido 15: home=1.0, away=0.0, draw=0.0
Partido 16: home=1.0, away=0.0, draw=-0.0
Partido 17: home=-0.0, away=0.9999999999999987, draw=0.0
Partido 18: home=0.0, away=1.0, draw=-0.0
Partido 19: home=0.0, away=1.0, draw=0.0
Partido 20: home=1.0, away=0.0, draw=-0.0
Partido 21: home=0.0, away=1.0, draw=-0.0
Partido 22: home=1.0000000000000053, away=0.0, draw=-0.0
Partido

## Le alcanza con conseguir 4 puntos?

In [5]:
crear_y_correr_modelo(forzar_proximos_partidos_dict={2: 'win', 8: 'draw'})

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve eliminated 17 rows and 25 columns.
MIP Presolve modified 2016 coefficients.
Reduced MIP has 545 rows, 189 columns, and 1241 nonzeros.
Reduced MIP has 180 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (1.21 ticks)
Found incumbent of value 0.000000 after 0.02 sec. (1.48 ticks)
Probing time = 0.00 sec. (0.88 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 545 rows, 189 columns, and 1241 nonzeros.
Reduced MIP has 180 binaries, 9 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.92 ticks)
Probing time = 0.00 sec. (0.88 ticks)
Clique table members: 32.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 16 threads.
Root relaxation solution time = 0.00 sec. (6.06 ticks)

        Nodes                             

Default row names c1, c2 ... being created.


Solución óptima: 28.0
Partido 1: home=0.0, away=-0.0, draw=1.0
Partido 2: home=0.0, away=1.0, draw=0.0
Partido 3: home=1.0, away=0.0, draw=-0.0
Partido 4: home=0.0, away=-0.0, draw=1.0
Partido 5: home=0.0, away=0.0, draw=1.0
Partido 6: home=1.0, away=-0.0, draw=-0.0
Partido 7: home=1.0, away=-0.0, draw=0.0
Partido 8: home=0.0, away=0.0, draw=1.0
Partido 9: home=0.0, away=1.0, draw=-0.0
Partido 10: home=1.0, away=-0.0, draw=-0.0
Partido 11: home=-0.0, away=1.0, draw=-0.0
Partido 12: home=1.0, away=0.0, draw=-0.0
Partido 13: home=1.0, away=0.0, draw=-0.0
Partido 14: home=1.0, away=-0.0, draw=0.0
Partido 15: home=1.0, away=0.0, draw=0.0
Partido 16: home=1.0, away=-0.0, draw=-0.0
Partido 17: home=-0.0, away=1.0, draw=-0.0
Partido 18: home=-0.0, away=1.0, draw=-0.0
Partido 19: home=0.0, away=1.0, draw=0.0
Partido 20: home=-0.0, away=1.0, draw=-0.0
Partido 21: home=1.0, away=-0.0, draw=-0.0
Partido 22: home=1.0, away=-0.0, draw=-0.0
Partido 23: home=1.0, away=0.0, draw=0.0
Partido 24: home=-

In [6]:
crear_y_correr_modelo(forzar_proximos_partidos_dict={2: 'draw', 8: 'win'})

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve eliminated 17 rows and 25 columns.
MIP Presolve modified 2016 coefficients.
Reduced MIP has 545 rows, 189 columns, and 1241 nonzeros.
Reduced MIP has 180 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (1.21 ticks)
Found incumbent of value 0.000000 after 0.02 sec. (1.48 ticks)
Probing time = 0.00 sec. (0.88 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 545 rows, 189 columns, and 1241 nonzeros.
Reduced MIP has 180 binaries, 9 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (1.00 ticks)
Probing time = 0.00 sec. (0.88 ticks)
Clique table members: 32.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 16 threads.
Root relaxation solution time = 0.00 sec. (6.23 ticks)

        Nodes                             

Default row names c1, c2 ... being created.


Solución óptima: 28.0
Partido 1: home=1.0, away=0.0, draw=-0.0
Partido 2: home=0.0, away=0.0, draw=1.0
Partido 3: home=0.0, away=1.0, draw=-0.0
Partido 4: home=1.0, away=-0.0, draw=-0.0
Partido 5: home=-0.0, away=1.0, draw=-0.0
Partido 6: home=-0.0, away=1.0, draw=-0.0
Partido 7: home=-0.0, away=1.0, draw=-0.0
Partido 8: home=1.0, away=0.0, draw=0.0
Partido 9: home=1.0, away=0.0, draw=0.0
Partido 10: home=1.0, away=-0.0, draw=0.0
Partido 11: home=0.0, away=1.0, draw=0.0
Partido 12: home=-0.0, away=1.0, draw=0.0
Partido 13: home=1.0, away=-0.0, draw=0.0
Partido 14: home=1.0, away=-0.0, draw=-0.0
Partido 15: home=1.0, away=0.0, draw=0.0
Partido 16: home=1.0, away=0.0, draw=-0.0
Partido 17: home=1.0, away=-0.0, draw=0.0
Partido 18: home=1.0, away=0.0, draw=-0.0
Partido 19: home=0.0, away=1.0, draw=0.0
Partido 20: home=1.0, away=0.0, draw=0.0
Partido 21: home=0.0, away=0.0, draw=1.0
Partido 22: home=1.0, away=-0.0, draw=-0.0
Partido 23: home=1.0, away=0.0, draw=0.0
Partido 24: home=1.0, aw

## Y con 6 puntos?

In [9]:
crear_y_correr_modelo(forzar_proximos_partidos_dict={2: 'win', 8: 'win'})

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve eliminated 354 rows and 82 columns.
MIP Presolve modified 2016 coefficients.
Reduced MIP has 208 rows, 132 columns, and 554 nonzeros.
Reduced MIP has 124 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (1.03 ticks)
Found incumbent of value 0.000000 after 0.02 sec. (1.21 ticks)
Probing time = 0.00 sec. (0.20 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 208 rows, 132 columns, and 554 nonzeros.
Reduced MIP has 124 binaries, 8 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.42 ticks)
Probing time = 0.00 sec. (0.20 ticks)
Clique table members: 32.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 16 threads.
Root relaxation solution time = 0.00 sec. (1.77 ticks)

        Nodes                              

Default row names c1, c2 ... being created.


Solución óptima: 7.0
Partido 1: home=1.0, away=0.0, draw=-0.0
Partido 2: home=0.0, away=1.0, draw=0.0
Partido 3: home=-0.0, away=1.0, draw=-0.0
Partido 4: home=1.0, away=0.0, draw=-0.0
Partido 5: home=-0.0, away=1.0, draw=0.0
Partido 6: home=1.0, away=-0.0, draw=-0.0
Partido 7: home=0.0, away=1.0, draw=-0.0
Partido 8: home=1.0, away=0.0, draw=0.0
Partido 9: home=-0.0, away=1.0, draw=-0.0
Partido 10: home=1.0, away=0.0, draw=-0.0
Partido 11: home=1.0, away=0.0, draw=-0.0
Partido 12: home=1.0, away=0.0, draw=-0.0
Partido 13: home=1.0, away=-0.0, draw=-0.0
Partido 14: home=-0.0, away=1.0, draw=0.0
Partido 15: home=1.0, away=0.0, draw=0.0
Partido 16: home=1.0, away=-0.0, draw=-0.0
Partido 17: home=1.0, away=-0.0, draw=0.0
Partido 18: home=-0.0, away=1.0, draw=-0.0
Partido 19: home=0.0, away=1.0, draw=0.0
Partido 20: home=1.0, away=0.0, draw=-0.0
Partido 21: home=1.0, away=-0.0, draw=-0.0
Partido 22: home=1.0, away=-0.0, draw=-0.0
Partido 23: home=1.0, away=0.0, draw=0.0
Partido 24: home=1.

### Y con 10?

In [8]:
crear_y_correr_modelo(forzar_proximos_partidos_dict={2: 'win', 8: 'win', 15: 'win', 19: 'draw'})

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve eliminated 482 rows and 107 columns.
MIP Presolve modified 2016 coefficients.
Reduced MIP has 80 rows, 107 columns, and 284 nonzeros.
Reduced MIP has 100 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (1.00 ticks)
Found incumbent of value 0.000000 after 0.00 sec. (1.14 ticks)
Probing time = 0.00 sec. (0.07 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 80 rows, 107 columns, and 284 nonzeros.
Reduced MIP has 100 binaries, 7 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.23 ticks)
Probing time = 0.00 sec. (0.07 ticks)
Clique table members: 31.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 16 threads.
Root relaxation solution time = 0.00 sec. (0.38 ticks)

        Nodes                               

Default row names c1, c2 ... being created.


Solución óptima: 0.0
Partido 1: home=1.0, away=0.0, draw=0.0
Partido 2: home=0.0, away=1.0, draw=0.0
Partido 3: home=1.0, away=0.0, draw=0.0
Partido 4: home=1.0, away=0.0, draw=0.0
Partido 5: home=1.0, away=0.0, draw=0.0
Partido 6: home=1.0, away=0.0, draw=0.0
Partido 7: home=1.0, away=0.0, draw=0.0
Partido 8: home=1.0, away=0.0, draw=0.0
Partido 9: home=0.0, away=1.0, draw=0.0
Partido 10: home=1.0, away=0.0, draw=0.0
Partido 11: home=1.0, away=0.0, draw=0.0
Partido 12: home=1.0, away=0.0, draw=0.0
Partido 13: home=1.0, away=0.0, draw=0.0
Partido 14: home=0.0, away=1.0, draw=0.0
Partido 15: home=0.0, away=1.0, draw=0.0
Partido 16: home=1.0, away=0.0, draw=0.0
Partido 17: home=1.0, away=0.0, draw=0.0
Partido 18: home=1.0, away=0.0, draw=0.0
Partido 19: home=0.0, away=0.0, draw=1.0
Partido 20: home=0.0, away=1.0, draw=0.0
Partido 21: home=1.0, away=0.0, draw=0.0
Partido 22: home=1.0, away=0.0, draw=0.0
Partido 23: home=1.0, away=0.0, draw=0.0
Partido 24: home=1.0, away=0.0, draw=0.0
Part